# Part 2

For the second part, we introduce you a task consisting in extracting information (entity tagging and classification) from images of products. You can focus on the nutritional facts as in the case of EDA. Your main goal is to develop pseudo code for the data processing and ML solution pipeline to automate the extraction. Please create a remote git repository with your pseudocode. It needs to illustrate the solution strategy and demonstrate good code practices. There is no need to implement fully working code or ML models. We will conduct a set of questions based on your solution and the repository created.

Requested functionalities:

o Ability to split data in multiple custom ways to train and evaluate the system

o Ability to handle different Machine Learning tasks: classification of single-label data objects, classification multi-label data objects, entity tagging.

o Ability to load, pre-process multimodal data (image and text) o Ability to handle the data in a PyTorch data loader

o Ability to define the model training and inference stages for the tasks (you can approach it as single or multitask model)

## Summary Tasks:


1. Single-Label Classification: Classifying Product Category
- Task: Given an image of a product, classify it into one of several categories (e.g., yogurt, cheese, biscuits).

- Input: Images of products, loaded from df['image_url'].
- Target Output: A single category label for each product, obtained from df['main_category_en'].
- Dataset Requirements:
    - The dataset should have non-empty values for df['image_url'] (the image of the product).
    - It should also have non-empty values for df['main_category_en'] (the product category label).


2. Multi-Label Classification: Predicting Nutritional Labels (High Calories, High Fat, High Protein)
- Task: Given an image of a product's nutritional table, classify whether the product is high in calories, high in fat, and/or high in protein. This is a multi-label classification problem because a product can belong to more than one category.

- Input: Images of nutritional tables, loaded from df['image_nutrition_url'].
- Target Output: Three binary labels:
    - High_calories: Whether the product is high in calories (binary: 0 or 1).
    - High_fat: Whether the product is high in fat (binary: 0 or 1).
    - High_protein: Whether the product is high in protein (binary: 0 or 1).
- Dataset Requirements:
    - Non-empty values for df['image_nutrition_url'] (the image of the nutritional table).
    - Ensure the nutritional values are present (e.g., fat_100g, sugars_100g, proteins_100g), which are used to generate the binary labels.

3. Entity Tagging: Labeling Ingredients in Product Ingredients Images
- Task: Perform entity tagging (Named Entity Recognition - NER) on the ingredients list provided in the product ingredients description image. The first problem here is to extract the text and for that we decided to use an OCR reader (did not implement it since it was straightfroward with the use of some already implemented APIs). Once we have the text extracted, the goal is to identify and label different entities in the text such as ingredients, amounts, and other relevant tokens. [Note: we used the ingredients description in text directly to avoid the text extraction step, however, using an OCR reader would be straightforward]

- Input: A textual description of the ingredients list, loaded from df['ingredients_text']. This is typically a string representing the ingredients of the product.

- Target Output: Entity labels for each token (word) in the ingredients text. For example:

    - B-ING: Beginning of an ingredient entity.
    - I-ING: Continuation of an ingredient entity.
    - B-AMT: Beginning of an amount entity (such as a percentage or quantity).
    - O: Token that is outside of any entity (i.e., not relevant to ingredients or amounts).

- Dataset Requirements:

    - The dataset must have non-empty entries in df['ingredients_text'] to provide valid text for the entity tagging task.
    - Rows with missing or NaN values in the ingredients_text column should be dropped to ensure only valid texts are used.

### Config file (.yaml)

Following Javier's indication from the previous meeting I decided to create a .yaml file to set the configuration. This allows for a centralized and flexible management of various configuration settings such as hyperparameters tuning.

## Functions 

In [47]:
import yaml
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
import requests
from io import BytesIO

# Load configuration from config file (.yaml)
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    return config

# Load the config file
config = load_config('config.yaml')

# Validation Image URL: we discard the urls that do not work
def filter_valid_urls(df, column, timeout=15):
    valid_rows = []
    for idx, row in df.iterrows():
        image_url = row[column]
        print(f"Checking URL: {image_url}")
        try:
            response = requests.get(image_url, timeout=timeout)
            response.raise_for_status()  
            valid_rows.append(row)  # Able to access the url
        except requests.exceptions.RequestException as e:
            print(f"Skipping URL due to error: {image_url}, Error: {e}") #We skip the urls that have access problems
    
    # Create a new dataframe with only valid urls
    return pd.DataFrame(valid_rows)

# Data Preprocessing
def load_and_preprocess_image(image_url):
    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))

        # Convert image to RGB if it is not already in RGB format (format that Resnet has to process images)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        preprocess_transform = transforms.Compose([
            transforms.Resize(tuple(config['data']['image_size'])),
            transforms.ToTensor(),
        ])
        return preprocess_transform(image)
    except Exception as e:
        print(f"Failed to load image from URL: {image_url}, Error: {e}")
        return None  # Handle invalid images

def preprocess_text(text):
    return text.lower().split()

# Custom Dataset class for task 2: here we select the features we want to include to each task (single-label and multi-label classification)
class Task2CustomDataset(Dataset):
    def __init__(self, dataframe, task_type='single_label_classification', transform=None):
        self.dataframe = dataframe
        self.task_type = task_type
        self.transform = transform
        
        # Assign an index to each category: dictionary
        if task_type == 'single_label_classification':
            self.single_label_mapping = {category: idx for idx, category in enumerate(dataframe['main_category_en'].unique())}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        if self.task_type == 'single_label_classification':
            image = load_and_preprocess_image(row['image_url'])
            if image is not None:
                # Use the dict to map the category to its corresponding index
                label = self.single_label_mapping[row['main_category_en']]
                return image, label

        elif self.task_type == 'multi_label_classification':
            image = load_and_preprocess_image(row['image_nutrition_url'])
            label = torch.tensor([row['High_Fat'], row['High_Sugar'], row['High_Protein']], dtype = torch.float)
            return image, label


# Data splitting
def split_data(dataframe):
    train_df, temp_df = train_test_split(dataframe, test_size=config['data']['split_ratios']['val'] + config['data']['split_ratios']['test'], random_state=config['data']['seed'])
    val_df, test_df = train_test_split(temp_df, test_size=config['data']['split_ratios']['test'] / (config['data']['split_ratios']['val'] + config['data']['split_ratios']['test']), random_state=config['data']['seed'])
    return train_df, val_df, test_df

# Function to create the 3 dataloaders: train, val, test loaders
def create_dataloaders(train_df, val_df, test_df, task_type):
    train_dataset = Task2CustomDataset(train_df, task_type=task_type)
    val_dataset = Task2CustomDataset(val_df, task_type=task_type)
    test_dataset = Task2CustomDataset(test_df, task_type=task_type)
    

    ### Had to avoid using num_workers because it wasnt compatible with my laptop

    # train_loader = DataLoader(train_dataset, batch_size=config['data']['batch_size'], shuffle=True, num_workers=config['data']['num_workers'])
    # val_loader = DataLoader(val_dataset, batch_size=config['data']['batch_size'], shuffle=False, num_workers=config['data']['num_workers'])
    # test_loader = DataLoader(test_dataset, batch_size=config['data']['batch_size'], shuffle=False, num_workers=config['data']['num_workers'])

    train_loader = DataLoader(train_dataset, batch_size=config['data']['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['data']['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config['data']['batch_size'], shuffle=False)
    
    return train_loader, val_loader, test_loader

# Evaluation: accuracy calculation
def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            if images is not None:
                images, labels = images.to(device), labels.to(device)  # Move images and labels to device
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')
    return accuracy

# Training
def train_model(model, train_loader, val_loader, device, criterion, optimizer, epochs=10):
    model.to(device)  # Move the model to the device
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            if images is not None and labels is not None:
                images, labels = images.to(device), labels.to(device)  # Move images and labels to the device
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
        print(f"Validation set accuracy: {evaluate_model(model, val_loader)}")


def evaluate_model(model, test_loader, device):
    compute_accuracy(model, test_loader, device= device)

## Classification models

### Description:

- Backbone: ResNet-18, a deep convolutional neural network (CNN) architecture that was designed to enable training very deep networks by introducing residual connections, is used as the backbone in both models due to its strong balance of performance and efficiency. Reasons why I chose Resnet18:

    - Pretrained Model: ResNet-18 is widely available as a pretrained model on large-scale datasets like ImageNet, providing a strong feature extraction capability. Additionally, pretrained models often generalize well to new tasks.

    - Residual Connections: The main innovation in ResNet is its use of skip connections, which help mitigate the vanishing gradient problem and allow the network to train effectively even with many layers. This enables deep learning models to have improved performance by making it easier to propagate gradients during backpropagation.

    - Computational Efficiency: ResNet-18, compared to deeper variants like ResNet-50 or ResNet-152, strikes a good balance between computational cost and accuracy. Having into account that had to run everything locally (just making sure the code was working - running at least) I decided to use the ResNet18.


- Use of Sigmoid for Multi-Label Classification: the sigmoid activation function is used at the final layer of the network to convert the raw logits into probabilities. In multi-label classification, each label (e.g., high-calories, high-fat, high-protein) is treated as a separate binary classification problem. Sigmoid outputs a probability for each label independently, allowing the model to predict multiple labels for a single input. The sigmoid function maps the logits to values between 0 and 1, which can be interpreted as the probability that each label is present.

- We decided to use Linear layers as the heads for each task after backbone due to efficiency and simplicity. 

In [37]:
import torch
import torch.nn as nn
import torchvision.models as models
import os
import torch.nn.functional as F

class SingleLabelClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(SingleLabelClassificationModel, self).__init__()
        # Load pretrained ResNet-18 as the backbone: using hf_cache in stored in current folder (easier to clean cache)
        cache_path = './hf_cache/'
        if not os.path.exists(cache_path):
            os.makedirs(cache_path)

        torch.hub.set_dir(cache_path)


        self.backbone = models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)
    

class MultiLabelClassificationModel(nn.Module):
    def __init__(self, num_labels):
        super(MultiLabelClassificationModel, self).__init__()
        # Load pretrained ResNet-18 as the backbone: using hf_cache in stored in current folder (easier to clean cache)
        cache_path = './hf_cache/'
        if not os.path.exists(cache_path):
            os.makedirs(cache_path)

        torch.hub.set_dir(cache_path)

        self.backbone = models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_labels)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        logits = self.backbone(x)
        return self.sigmoid(logits)


## Single-label classification

1. Read the dataset and filter to ensure valid urls
2. Split training, validation, test sets -> (PyTorch) Dataloeaders
3. Model initialisation: resnet18 as backbone and Linear Layer as classification head
4. We add torch device to enable the use of GPUs when possible
4. Model training: CrossEntropyLoss is used for training using Adam optimizer


In [42]:
nrows = 10000

# Read dataframe
df = pd.read_csv(config['data']['dataset_paths']['dataset_csv'], delimiter='\t',on_bad_lines='skip', low_memory=False, nrows=nrows)

column_single_label_image = 'image_url'
df_classification = df[df[column_single_label_image].notna() & df['main_category_en'].notna()]


# Split the dataset: 0.7 training, 0.15 val, 0.15 test
train_df, val_df, test_df = split_data(df_classification)

# DataLoaders
train_loader, val_loader, test_loader = create_dataloaders(train_df, val_df, test_df, task_type='single_label_classification')

criterion_single = torch.nn.CrossEntropyLoss()

num_classes = df_classification['main_category_en'].nunique()
single_label_model = SingleLabelClassificationModel(num_classes)

# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(single_label_model.parameters(), lr=config['model']['learning_rate'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train_model(single_label_model, train_loader, val_loader, criterion, optimizer, device = device, epochs = config['model']['epochs'])


## Multi-label classification

1. Read the dataset and filter to ensure valid urls
2. Filtering: non-missing values for the nutritional columns (fat_100g, sugars_100g, proteins_100g). Nutritional values transformed into binary labels -> *Multi-label classification*
2. Split training, validation, test sets -> (PyTorch) Dataloeaders
3. Model initialisation: resnet18 as backbone and Linear Layer as classification head
4. We add torch device to enable the use of GPUs when possible
5. Model training: Binary Cross-Entropy with Logits Loss is used for training with Adam optimiser

In [ ]:
# We dont read dataframe the dataframe again in order to use the same we were using for single-label classification

nutrition_columns = ['fat_100g', 'sugars_100g', 'proteins_100g']
column_multi_label_image = 'image_nutrition_url'


# Data preprocessing: valid urls + Rows drop with missing nutritional values (fat, sugars, and proteins)
df_multi_classification = df[df[column_multi_label_image].notna()]
df_filtered = df_multi_classification.dropna(subset=nutrition_columns)


# Nutritional values -> binary labels (thresholds should be discussed with the team or client)
df_filtered['High_Fat'] = pd.to_numeric((df_filtered[nutrition_columns[0]] > 10).astype(int), errors='coerce')
df_filtered['High_Sugar'] = pd.to_numeric((df_filtered[nutrition_columns[1]] > 5).astype(int), errors='coerce')
df_filtered['High_Protein'] = pd.to_numeric((df_filtered[nutrition_columns[2]] > 7.5).astype(int), errors='coerce')
num_labels = len(nutrition_columns)

# Split the dataset: 0.7 training, 0.15 val, 0.15 test
train_df, val_df, test_df = split_data(df_filtered)

# Dataloaders
train_loader, val_loader, test_loader = create_dataloaders(train_df, val_df, test_df, task_type='multi_label_classification')


criterion_multi = torch.nn.BCEWithLogitsLoss()
multi_label_model = MultiLabelClassificationModel(num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train_model(single_label_model, train_loader, val_loader, criterion, optimizer, device = device, epochs = config['model']['epochs'])


## Entity tagging

- Task: Perform entity tagging (Named Entity Recognition - NER) on the ingredients list provided in the product ingredients description image. The first problem here is to extract the text and for that we decided to use an OCR reader (did not implement it since it was straightfroward with the use of some already implemented APIs). Once we have the text extracted (we used 'ingredients_text' column where we had this exact info), the goal is to identify and label different entities in the text such as ingredients, amounts, and other relevant tokens. [Note: we used the ingredients description in text directly to avoid the text extraction step, however, using an OCR reader would be straightforward]


We want to implement a supervised learning model, and for that we need to have ground truth data to compare model's output with the ground truth and then adjust the model's parameters accordingly. Since we do not have grounf truth data available it may require manual labeling. Manual labeling for tasks like entity tagging is time-consuming and labor-intensive. Typically, domain experts or annotators would need to go through each product’s ingredient list and manually tag ingredients and amounts. I decided to use the openai api to avoid wasting time doing the manuak labeling. We used the model GPT-4 which is asked to generate a labeled dataset quickly. The output is used to train machine learning model, in this case the BERT transformer.

(Other way which is less computationally intensive and more efficient to solve this task is just using the output of the openai api. However, we would need to explore how the openai model performs and see if it is trustworthy)

Setup
1. Loading config file (.yaml) and using openai_api_key (Config class)
2. Split training, validation, test sets -> (PyTorch) Dataloeaders
3. Ground truth collection: using openai api

### Configuration and other auxilary functions

In [ ]:
# Import necessary libraries
import openai
import yaml
from sklearn.model_selection import train_test_split

# Load configuration using config.yaml and the class Config to use the openai_api_key
def load_config(config_path='config.yaml'):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

# Configuration class to manage the loaded configuration
class Config:
    def __init__(self, config_path='config.yaml'):
        self.config = load_config(config_path)
        openai.api_key = 'openai_api_key'  # Add your OpenAI API key here
    
    def get(self, key):
        return self.config.get(key)

# Function to split the data according to config.yaml ratios
def split_data(dataframe, config):
    train_ratio = config['data']['split_ratios']['train']
    val_ratio = config['data']['split_ratios']['val']
    test_ratio = config['data']['split_ratios']['test']
    
    train_df, test_df = train_test_split(dataframe, test_size=(val_ratio + test_ratio), random_state=config['data']['seed'])
    val_df, test_df = train_test_split(test_df, test_size=test_ratio / (test_ratio + val_ratio), random_state=config['data']['seed'])
    
    return train_df, val_df, test_df

# Function to label ingredients using OpenAI API
def label_ingredients(text):
    # OpenAI API call for entity tagging/classification
    response = openai.Completion.create(
        model="gpt-4",
        prompt=f"Label the following text using BIO tagging: {text}. Use the following tags: 'B-ING' for beginning of an ingredient, 'I-ING' for continuation of an ingredient, 'B-AMT' for amount, 'O' for other tokens.",
        max_tokens=100
    )
    # Parse and return response
    return response['choices'][0]['text']

# Function to preprocess and label dataset using OpenAI API
def preprocess_and_label(dataframe):
    dataframe['entity_tags'] = dataframe['ingredients_text'].apply(label_ingredients)
    return dataframe

### Custom new dataset and creating dataloader

1. IngredientsDataset is a class to create the PyTorch Dataset needed for entity tagging task:
- The input text (ingredients) is found in the ingredients_text column.
- The ground truth BIO tags are in the entity_tags column, labeled by the OpenAI API. (Maybe other approaches can be used here like manual labeling)
- The data is tokenized by splitting the input text into words (tokens) in the same way as the ground truth tags, with one token per word
- It handles padding and truncation to ensure that each sequence is the same length (max_len)
- Converts the tokens to input IDs (using a tokenizer) and creates an attention mask (where [PAD] tokens are ignored)

2. DataLoader creation



In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Custom Dataset -> Entity Tagging
class IngredientsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, label_to_id):
        self.data = dataframe['ingredients_text'].dropna().reset_index(drop=True)
        self.labels = dataframe['entity_tags'].dropna().reset_index(drop=True)  # Ground truth saved in entity_tags column (labeled with OpenAI)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_to_id = label_to_id  # Mapping of BIO labels to integers -> label_to_id = {'B-ING': 0, 'I-ING': 1, 'B-AMT': 2, 'O': 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #Tokenize text in the same way as the ground truth (each word = 1 token)
        text = self.data[idx]
        labels = self.labels[idx].split()  # Assume labels are space-separated for each token -> 'B-ING I-ING O B-AMT O B-ING I-ING O B-ING O B-AMT'
        tokens = text.split()

        # Truncate -> max length
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
            labels = labels[:self.max_len]
        else:
            padding_len = self.max_len - len(tokens)
            tokens += ['[PAD]'] * padding_len
            labels += ['O'] * padding_len 

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1 if token != '[PAD]' else 0 for token in tokens]
        label_ids = [self.label_to_id[label] for label in labels]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(label_ids, dtype=torch.long)
        }
    
    
# PyTorch DataLoader
def create_dataloader(dataframe, batch_size, num_workers):
    dataset = IngredientsDataset(dataframe)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return dataloader

### Creating Model -> BERT and its functions(train, inference)
Why BERT?
- BERT: transformer-based model pre-trained on massive amounts of text data and learns contextual representations of words by considering both their left and right context (bidirectional). Really useful for this task
- Use of 'bert-base-uncased' model: benefits from a strong understanding of language structure and semantics
- We fine-tune the model to predict BIO entity tags for each word in the sequence (ingredients text)
- Can handle well token-level classification, each word in the text might belong to a different category

In [ ]:
import torch
from transformers import BertForSequenceClassification

bio_labels = ['B-ING', 'I-ING', 'B-AMT', 'O']

class TransformerModel(nn.Module):
    def __init__(self, config):
        super(TransformerModel, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels= len(bio_labels))  # 3 classes: ingredient, quantity, other

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits
    
def train_model(dataloader, model, optimizer, loss_fn, config, device):
    model.train()
    for epoch in range(config['model']['epochs']):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # Assuming labels are provided
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{config['model']['epochs']}, Loss: {total_loss/len(dataloader)}")

# Model Inference
def inference(model, dataloader, device):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
    return all_preds

### Execution

1. Read the dataset and filter to ensure non empty ingredients_text
2. Split training, validation, test sets -> (PyTorch) Dataloeaders
3. Model initialisation: BERT model 
4. We add torch device to enable the use of GPUs when possible
4. Model training: Cross-Entropy Loss is used for training with Adam optimiser

In [ ]:
# Read data file
nrows = 10000 
df = pd.read_csv(config['data']['dataset_paths']['dataset_csv'], delimiter='\t', on_bad_lines='skip', low_memory=False, nrows=nrows)

# Preprocess data: filter out NaNs
df = df.dropna(subset=['ingredients_text'])
    
# Label data addition to the dataframe using OpenAI API
df = preprocess_and_label(df)

label_to_id = {'B-ING': 0, 'I-ING': 1, 'B-AMT': 2, 'O': 3}
# Split the dataset: 0.7 training, 0.15 val, 0.15 test
train_df, val_df, test_df = split_data(df, config)

# Create DataLoaders
train_loader = create_dataloader(train_df, config['data']['batch_size'], config['data']['num_workers'])
val_loader = create_dataloader(val_df, config['data']['batch_size'], config['data']['num_workers'])
test_loader = create_dataloader(test_df, config['data']['batch_size'], config['data']['num_workers'])
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = len(label_to_id)  
model = TransformerModel(config).to(device)
        
optimizer = optim.Adam(model.parameters(), lr=config['model']['learning_rate'])
loss_fn = nn.CrossEntropyLoss()  

# Train the model
train_model(train_loader, model, optimizer, loss_fn, config, device)
